In [2]:
#Import des dépendences
import pandas as pd 
import vaex as vx

from datetime import datetime

In [3]:
df = vx.from_csv("https://www.afnic.fr/data/opendata/202004_OPENDATA_A-NomsDeDomaineEnPointFr.zip",  delimiter=";", encoding='latin1', header=0, index_col=None)

In [4]:
%%time
for name in df.column_names: #Personne n'aime les espace dans les headers ni les accents 
    df.rename(name, name.replace('é', 'e').lower())

#On prépare une colonne avec une date correcte
def from_french_date(d):
    try:
        return datetime.strptime(d, '%d-%m-%Y').strftime('%Y-%m-%d')
    except ValueError:
        return None
    
df.create_at = df.date_de_creation.apply(from_french_date).astype('datetime64[ns]')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.23 ms


In [5]:
%%time
df_filtered = df[df.create_at.dt.year >= 2020]

CPU times: user 15.6 ms, sys: 15.6 ms, total: 31.2 ms
Wall time: 3.77 ms


In [6]:
%%time
df_filtered.create_at = df_filtered.date_de_creation.apply(from_french_date).astype('datetime64[ns]')
df_filtered.delete_at = df_filtered.date_de_retrait_du_whois.apply(from_french_date).astype('datetime64[ns]')

CPU times: user 15.6 ms, sys: 0 ns, total: 15.6 ms
Wall time: 3.4 ms


In [7]:
%%time
df_filtered.select(df_filtered.date_de_retrait_du_whois.isnan(), name="actif")
df_filtered.select(df_filtered.date_de_retrait_du_whois.notnan(), name="inactif")
df_filtered.select(df_filtered.nom_de_domaine.str.contains('covid'), name='covid')
df_filtered.select((df_filtered.nom_de_domaine.str.contains('covid') and df_filtered.date_de_retrait_du_whois.isnan()), name='covid_actif')
df_filtered.select((df_filtered.nom_de_domaine.str.contains('covid') and df_filtered.date_de_retrait_du_whois.notnan()), name='covid_inactif')

CPU times: user 15.6 ms, sys: 0 ns, total: 15.6 ms
Wall time: 18.7 ms


In [8]:
%%time
df_filtered.count(selection=["covid", "actif", "inactif", "covid_actif", "covid_inactif"])

CPU times: user 4min 51s, sys: 1min 35s, total: 6min 27s
Wall time: 5min 19s


array([  1182, 208732,    847, 208732,    847])

In [9]:
%%time
df_filtered.groupby(df_filtered.date_de_creation, agg='count').sort('count', ascending=False)

CPU times: user 62.5 ms, sys: 15.6 ms, total: 78.1 ms
Wall time: 61.9 ms


#,date_de_creation,count
0,27-04-2020,2681
1,20-04-2020,2483
2,16-01-2020,2477
3,29-04-2020,2471
4,12-02-2020,2453
...,...,...
115,08-03-2020,894
116,14-03-2020,856
117,15-03-2020,820
118,22-02-2020,818


In [10]:
%%time
df_filtered.evaluate(df_filtered.nom_de_domaine, selection="covid").tolist()

CPU times: user 172 ms, sys: 0 ns, total: 172 ms
Wall time: 102 ms


['achat-masque-covid.fr',
 'achetermasquecovid19.fr',
 'acovid.fr',
 'actanticovid.fr',
 'action-groupe-covid-19.fr',
 'actions-fondations-covid.fr',
 'affichage-covid-19.fr',
 'affichage-covid19.fr',
 'affichagecovid19.fr',
 'affiche-covid-19.fr',
 'affiche-covid19.fr',
 'affichecovid19.fr',
 'affiches-covid-19.fr',
 'affichescovid19.fr',
 'after-covid.fr',
 'aftercovid.fr',
 'aftercovid-19.fr',
 'aftercovid19.fr',
 'agircovid.fr',
 'agir-covid-19.fr',
 'agir-covid19.fr',
 'agircovid19.fr',
 'aide-covid.fr',
 'aide-indemnisation-salaries-covid19.fr',
 'aide-indemnisation-soignants-covid19.fr',
 'aide-parentale-scolaire-covid-19.fr',
 'aide-scolaire-parentale-covid-19.fr',
 'aides-entreprises-covid19.fr',
 'alertcovid.fr',
 'alertcovid19.fr',
 'alerte-covid.fr',
 'alertecovid.fr',
 'alerte-covid19.fr',
 'alertecovid19.fr',
 'allagainstcovid.fr',
 'allagainstcovid19.fr',
 'alliance-covid-19.fr',
 'alliance-covid19.fr',
 'alliancecovid19.fr',
 'alliancepro-covid19.fr',
 'allo-covid.fr',
